In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("openAI-community/gpt2")
tokenizer = AutoTokenizer.from_pretrained("openAI-community/gpt2")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
import torch

def generate(model, tokenizer, input_ids, max_length=50, do_sample=False, top_k=None):
    generated = input_ids

    model.eval()

    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(input_ids=input_ids)
            logits = outputs.logits

            if do_sample:
                if top_k is not None:
                    logits = top_k_logits(logits, top_k)
                next_token = torch.multinomial(torch.nn.functional.softmax(logits[:, -1, :], dim=-1), num_samples=1)
            else:
                next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)

            generated = torch.cat((generated, next_token), dim=1)

            if next_token.item() == tokenizer.eos_token_id:
                break

            input_ids = generated

    return generated

def top_k_logits(logits, k):
    values, _ = torch.topk(logits, k)
    min_values = values[:, -1].unsqueeze(-1)
    return torch.where(logits < min_values, torch.full_like(logits, -float('Inf')), logits)

In [24]:
import torch

prompts = "hey "
encoding = tokenizer(prompts, return_tensors="pt")
input_ids = encoding['input_ids']

model.eval()

generated_ids = generate(model=model, tokenizer=tokenizer, input_ids=input_ids,do_sample=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generated_text)


hey?"

On that note.. remember remember Kerry being the first country that picked his cable news show out of the 1000st Days of Helping Kids In Colorado rodeo where the Johnny Varela juggle students, scientists and audience members became the
